In [180]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

X, y = load_iris(return_X_y=True)

In [181]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.33, random_state=42)

In [6]:
# Todo : Adding the input of feature selction method
# Todo ; Adding the input of bootstap sample number

In [168]:
def gini_index(*groups):
    gini_value = 0
    total_num = np.sum([group[0].shape[0] for group in groups])

    for group in groups:
        y = group[1]
        group_size = y.shape[0]

        _, count = np.unique(y, return_counts=True)
        p = count/group_size
        weight = group_size/total_num

        gini_value += (1- np.sum(p**2)) * weight
    return gini_value


def split(X, y, split_value):
    mask = X < split_value
    group_1 = (X[mask], y[mask])
    group_2 = (X[mask == 0], y[mask == 0])
    return group_1, group_2

def best_split(X, y, m = None):
    spliting_list = []
    feature_index = np.arange(X.shape[1])
    
    if m != None:
        feature_index = np.random.choice(feature_index, 2, replace=False)
        
    for i in feature_index:
        split_range = np.unique(X[:,i])
        for split_value in split_range:
            split_group = split(X[:, i], y, split_value)
            if check_legal_split(*split_group, min_samples_leaf=1):
                spliting_list.append([gini_index(*split_group), split_value, i])
               
    if len(spliting_list) ==0:
#         print('No Data can be split')
        return np.NaN, np.NaN, None, None
    else:
        optimal_split, _,_ = np.argmin(np.array(spliting_list), axis = 0)
        _, optimal_split_value, label = np.array(spliting_list[optimal_split])

        bool_mask = X[:, int(label)] < optimal_split_value
        group1 = (X[bool_mask], y[bool_mask])
        group2 = (X[bool_mask == 0 ], y[bool_mask == 0])
        return optimal_split_value, label, group1, group2

def check_legal_split(*groups, min_samples_leaf=1):
    for group in groups:
        if group[0].shape[0] < min_samples_leaf:
            return False
        else:
            return True
        
def pprint_tree(node, file=None, _prefix="", _last=True):
    print(_prefix, "`- " if _last else "|- ", "Depth :{}, Split Feature:{}, Split Value:{}, Label:{}".format(node.depth,node.split_value, node.split_feature, node._label), sep="", file=file)
    _prefix += "   " if _last else "|  "
    
    if node._left_child == None or node._right_child == None:
        return None
    
    pprint_tree(node._left_child, file, _prefix, _last = False)
    pprint_tree(node._right_child, file, _prefix, _last = True)

In [271]:
class Node:
    def __init__(self, depth = 0):
        self.split_value = None
        self.split_feature = None
        self._label = None
        self._left_child = None
        self._right_child = None
        self._leaf = False
        self.depth = depth
    
    def train(self, X, y, max_depth=8, min_samples_split=2, min_samples_leaf=1):
        m = int(np.sqrt(X.shape[1]))
#         print('Depth', self.depth)
#         print('Sample', X.shape[0])
#         print('X ', X)
#         print('y ', y)

    
        self.split_value, self.split_feature, group1, group2 = best_split(X, y, m = m)
        
        if group1 == None and group2 == None:
            self.sprout(y)
        else:
            if self.depth < max_depth and X.shape[0] > min_samples_split and len(np.unique(X)) != len(X[0]):
                self._left_child = Node(self.depth + 1)
                self._right_child = Node(self.depth + 1)
                self._left_child.train(*group1, max_depth, min_samples_split, min_samples_leaf)
                self._right_child.train(*group2, max_depth, min_samples_split, min_samples_leaf)
            else:
                self.sprout(y)
            
    def sprout(self, y):
        self._leaf = True

        labels, count = np.unique(y, return_counts = True)
        self._label = labels[np.argmax(count)]
        
    def predict(self, x):
        if self._leaf == True:
            return self._label
        
        if x[int(self.split_feature)] < self.split_value:
            return self._left_child.predict(x)
        else:
            return self._right_child.predict(x)

class Tree:
    def __init__(self, max_depth=8, min_samples_split=2, min_samples_leaf=1):
        self._root = Node()
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        
    def train(self, X, y, bootstrap = False):
        
        if bootstrap == True:
            N = X_train.shape[0]
            X_range = np.arange(N)
            index = np.random.choice(X_range, N, replace=True)
            X = X[index]
            y = y[index]
        
        self._root.train(X, y,
                        self.max_depth, 
                        self.min_samples_split, 
                        self.min_samples_leaf)

    def predict(self, X):
        prediction = []
        for i in range(len(X)):
            prediction.append(self._root.predict(X[i]))
        return prediction
    
    def print_tree(self):
        pprint_tree(self._root)

class Forest:
    def __init__(self, max_depth=8, min_samples_split=2, min_samples_leaf=1, 
                 n_estimators = 100):
        
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.n_estimators = n_estimators
        self.trees = []
    
    def train(self, X, y, bootstrap = False, bootstrap_ratio = 0):
        for i in range(self.n_estimators):
            tree = Tree(max_depth=4)
            tree.train(X,y,bootstrap = True)
            self.trees.append(tree)
    
    def predict(self, X):
        trees_result = np.array([tree.predict(X) for tree in self.trees])
        unqiue_count = list(map(lambda x : np.unique(x, return_counts = True),trees_result.T))
        index = [np.argmax(x[1]) for x in unqiue_count]
        labels = [x[0] for x in unqiue_count]
        return np.array([label[indices] for label, indices in zip(labels, index)])


# Debug

In [144]:
X_test = np.array([[5.,  3.2, 1.2, 0.2],
                   [5.8, 4. , 1.2, 0.2],
                   [5.,  3.2 ,1.2, 0.2]])
y_test = np.array([0, 0, 0])

In [148]:
spliting_list = []
feature_index = np.arange(X.shape[1])

if m != None:
    feature_index = np.random.choice(feature_index, 2, replace=False)

for i in feature_index:
    split_range = np.unique(X[:,i])
    for split_value in split_range:
        split_group = split(X[:, i], y, split_value)
        if check_legal_split(*split_group, min_samples_leaf=1):
            spliting_list.append([gini_index(*split_group), split_value, i])

if len(spliting_list) ==0:
    print('No Data can be split')
#     return np.NaN, np.NaN, None, None
else:
    optimal_split, _,_ = np.argmin(np.array(spliting_list), axis = 0)
    _, optimal_split_value, label = np.array(spliting_list[optimal_split])

    bool_mask = X[:, int(label)] < optimal_split_value
    group1 = (X[bool_mask], y[bool_mask])
    group2 = (X[bool_mask == 0 ], y[bool_mask == 0])
#     return optimal_split_value, label, group1, group2

No Data can be split


# Check with sckit-learn

In [182]:
from sklearn.ensemble import RandomForestClassifier

In [183]:
clf = RandomForestClassifier(random_state=0,
                                 n_estimators=100)

In [184]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [185]:
skprediction = clf.predict(X_test)

In [281]:
forest = Forest()

In [282]:
forest.train(X_train, y_train)

In [283]:
trees_result = forest.predict(X_test)

In [285]:
accuracy_score(y_test, trees_result)

0.98

In [284]:
skprediction == trees_result 

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])